In [1]:
import pandas as pd
from datetime import datetime
import flair
from textblob import TextBlob

In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tqdm import tnrange, tqdm_notebook, tqdm

In [6]:
df = pd.read_excel('BTC_TWEETS.xlsx')
df.head()

,Unnamed: 0,Datetime,Tweet Id,Text,Username,hashtags,lang,likeCount,retweetCount
0,0,2022-01-01 23:59:58+00:00,1.477430e+18,@doejohn73 @Diegoqs80 El Bitcoin serÃ¡ manipul...,G181133,NaN,es,3,0
1,1,2022-01-01 23:59:35+00:00,1.477430e+18,If your interested in losing money in 2022 kee...,robert54177167,['bitcoin'],en,7,2
2,2,2022-01-01 23:59:26+00:00,1.477430e+18,@SirPantsALot2 @fairguinevere @totallynotem_ @...,AxolotolH,NaN,en,129,0
3,3,2022-01-01 23:59:10+00:00,1.477430e+18,BTC diamond reversal and expected volatility f...,nyan_lou618,"['Bitcoin', 'trading']",en,1,0
4,4,2022-01-01 23:59:10+00:00,1.477430e+18,if we ever get to the point where we start pay...,joybowtie,NaN,en,1,0


In [8]:
df['Date'] = pd.to_datetime(df['Datetime'])

In [9]:
df.drop('Unnamed: 0',axis=1,inplace=True)
df.drop('Tweet Id',axis=1,inplace=True)
df.drop('Username',axis=1,inplace=True)
df.drop('hashtags',axis=1,inplace=True)
df.drop('lang',axis=1,inplace=True)
df.drop('Datetime',axis=1,inplace=True)

In [10]:
df.dropna(inplace=True)

In [11]:
analyzer = SentimentIntensityAnalyzer()
compound=[]
pos_tweet = []
neg_tweet = []
neu_tweet = []
for i,j in enumerate(tqdm(df['Text'])):
    v = analyzer.polarity_scores(j)
    compound.append(v['compound'])
    pos_tweet.append(v['pos'])
    neg_tweet.append(v['neg'])
    neu_tweet.append(v['neu'])

df['compound'] = compound
df.head(2)

100%|████████████████████████████████████████████████████████████████████████| 634878/634878 [04:12<00:00, 2516.13it/s]


,Text,likeCount,retweetCount,Date,compound
0,@doejohn73 @Diegoqs80 El Bitcoin serÃ¡ manipul...,3,0,2022-01-01 23:59:58+00:00,0.0000
1,If your interested in losing money in 2022 kee...,7,2,2022-01-01 23:59:35+00:00,0.8225


In [12]:
df['pos'] = pos_tweet
df['neg'] = neg_tweet
df['neu'] = neu_tweet
df['compound'] = df['compound'].round(decimals=1)
df['pos'] = df['pos'].round(decimals=1)
df['neg'] = df['neg'].round(decimals=1)
df['neg'] = df['neg'].round(decimals=1)

In [13]:
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')
fmt = '%Y-%m-%d %H:00:00'

2022-05-03 22:57:57,898 loading file C:\Users\mpmur\.flair\models\sentiment-en-mix-distillbert_4.pt


In [14]:
def sentiment_val_flair(flair_tag,flair_score):
    neg = 'NEGATIVE' in flair_tag
    val = float(flair_score)
    if neg:
        return -val
    return val

In [15]:
def get_sentiment_report(df):
    tb_sentiment_polarity_ = []
    tb_sentiment_subjectivity_ = []
    flair_sentiment_ = []
    for i,j in enumerate(tqdm(df['Text'])):
        flair_s = flair.data.Sentence(j)
        flair_sentiment.predict(flair_s)
        flair_tag = flair_s.tag
        flair_score = flair_s.score
        flair_val = sentiment_val_flair(flair_tag,flair_score)
        flair_sentiment_.append(flair_val)
        tb_sentiment_polarity_.append(TextBlob(j).sentiment[0])
        tb_sentiment_subjectivity_.append(TextBlob(j).sentiment[1])
    df['Twitter_Flair'] = flair_sentiment_
    df['Twitter_TB_Polarity'] = tb_sentiment_polarity_
    df['Twitter_TB_Subjectivity'] = tb_sentiment_subjectivity_
    

In [16]:
get_sentiment_report(df)

100%|███████████████████████████████████████████████████████████████████████| 634878/634878 [18:34:14<00:00,  9.50it/s]


In [17]:
df.to_csv('BTC_TWEETS_PROCESSED.csv')